# Period-by-Period comparison

A basic step in data analysis is to make comparisons of different distributions (i.e. cuts, data sets, Data/MC). When defining a plotting style associative containers are a very useful tool to define styles and assign them dynamically to plotting objects. The task is to define a plotting style and store it for several data samples in an associative container. For the associative containers try to use the following containers:
    - std::map
    - ROOT's TMap

In [ ]:
class PlotStyle : public TObject {
        Color_t fColor;
        Style_t fMarker;
    public:
        PlotStyle() : TObject(), fColor(0), fMarker(0) {}
        PlotStyle(Color_t color, Style_t marker) : TObject(), fColor(color), fMarker(marker) {}
        virtual ~PlotStyle() {}
        
        void SetColor(Color_t color) { fColor = color; }
        void SetMarker(Style_t marker) { fMarker = marker; }
        
        Color_t GetColor() const { return fColor; }
        Style_t GetMarker() const { return fMarker; }
            
        template<typename t>
        void SetStyle(t * graphicsobject) const {
            graphicsobject->SetMarkerColor(fColor);
            graphicsobject->SetLineColor(fColor);
            graphicsobject->SetMarkerStyle(fMarker);
        }
};

Create the associative containers and fill them with styles. The following styles should be applied for the different datasets:
    
| Period | Color  | Marker |
|--------|--------|--------|
| LHC17h | Red    |   24   |
| LHC17i | Blue   |   25   |
| LHC17k | Green  |   26   |
| LHC17m | Violet |   27   |

In [ ]:
// Your code here
std::map<string, PlotStyle *> styles = {{"LHC17h", new PlotStyle(kRed, 24)}, {"LHC17i", new PlotStyle(kBlue, 25)},{"LHC17j", new PlotStyle(kGreen, 26)}, {"LHC17m", new PlotStyle(kViolet, 27)}};

Test your code: Plot the histogram named hPt for all periods mentiond above in one canvas.

In [ ]:
std::map<std::string, TH1 *> periodhists;
{
    std::unique_ptr<TFile> reader(TFile::Open("./data/periodcomparison.root"));
    for(auto p : TRangeDynCast<TKey>(gDirectory->GetListOfKeys())) {
        std::string period = p->GetName();
        reader->cd(period.data());
        auto pthist = static_cast <TH1 *>(gDirectory->Get("hpt"));
        pthist->SetDirectory(nullptr);
            
        PlotStyle *style(nullptr);
        // Your code here: find the style in the lookup table, set the style to the histogram
        style = styles[period];
            
        if(style){
            style->SetStyle<std::remove_pointer<decltype(pthist)>::type>(pthist); // Would this code work if you suddenly decide to change plotting object from TGraph to TH1 ?    
        }
        periodhists[period] = pthist;
    }
}
auto plot = new TCanvas("periodComparison", "Period comparison", 800, 600);
plot->Draw();
plot->SetLogy();
auto frame = new TH1F("frame", "; p_{t} (GeV/c); 1/N_{ev} dN/dp_{t}", 100, 0., 100.);
frame->SetDirectory(nullptr);
frame->SetStats(false);
frame->GetYaxis()->SetRangeUser(1e-10, 100);
frame->Draw();
auto leg = new TLegend(0.7, 0.65, 0.89, 0.89);
leg->SetBorderSize(0);
leg->SetFillStyle(0);
leg->SetTextFont(42);
leg->Draw();
for(const auto &en : periodhists) {
    en.second->Draw("epsame");
    leg->AddEntry(en.second, en.first.data(), "lep");
}
plot->cd();
plot->Update();